# MSADS 507 M4

In [1]:
import numpy as no
import pandas as pd
import pymysql as mysql
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

In [2]:
conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd='****',db='gravity_books')

In [3]:
tableNames = pd.read_sql("""SHOW TABLES""", conn) 
tableNames

,Tables_in_gravity_books
0,address
1,address_status
2,author
3,book
4,book_author
5,book_language
6,country
7,cust_order
8,customer
9,customer_address


In [4]:
# 1. Period: the transaction between ** to **

#latest order date:
a = pd.read_sql_query("""

SELECT order_date 
FROM cust_order
ORDER BY order_date DESC 
LIMIT 1

""" , con = conn) 
a

,order_date
0,2023-02-05 16:41:43


In [5]:
# earliest order date:

b = pd.read_sql_query("""

SELECT order_date 
FROM cust_order 
ORDER BY order_date
LIMIT 1

""" , con = conn) 
b

,order_date
0,2020-02-06 18:21:33


In [6]:
# order counts in the first year:

c = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2020-02-06' AND '2021-02-05'

""" , con = conn) 
c

,COUNT(*)
0,2550


In [7]:
# order counts in the second year:

d = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2021-02-06' AND '2022-02-05'

""" , con = conn) 
d

,COUNT(*)
0,2471


In [8]:
# order counts in the last year:

d = pd.read_sql_query("""

SELECT COUNT(*)
FROM cust_order
WHERE order_date BETWEEN '2022-02-06' AND '2023-02-05'

""" , con = conn) 
d

,COUNT(*)
0,2511


In [9]:
# 2. how many orders, and books are sold total

e = pd.read_sql_query("""

SELECT COUNT(order_id), COUNT(book_id) books_sold
FROM order_line;

""" , con = conn) 
e

,COUNT(order_id),books_sold
0,26950,26950


In [10]:
# 3. how many customers have

f = pd.read_sql_query("""

SELECT COUNT(DISTINCT(customer_id)) 
FROM cust_order;

""" , con = conn) 
f

,COUNT(DISTINCT(customer_id))
0,1681


In [11]:
# 4. Which customer placed most order

g = pd.read_sql_query("""

SELECT c.customer_id, c.first_name, c.last_name, COUNT(*) 
FROM customer c
LEFT JOIN cust_order co
ON c.customer_id = co.customer_id
GROUP BY c.customer_id
ORDER BY COUNT(DISTINCT( co.order_id)) DESC LIMIT 1;

""" , con = conn) 
g

,customer_id,first_name,last_name,COUNT(*)
0,650,Reuben,Usher,21


In [12]:
# 5. What shipping method is most preferred, and the percenrage of total

h = pd.read_sql_query("""

SELECT sm.method_name, COUNT(*),COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM shipping_method sm
RIGHT JOIN cust_order co
ON sm.method_id = co.shipping_method_id
GROUP BY sm.method_name
ORDER BY COUNT(*) DESC;

""" , con = conn) 
h

,method_name,COUNT(*),Percent of total
0,Priority,1938,25.66887
1,Express,1910,25.29801
2,International,1861,24.64901
3,Standard,1841,24.38411


In [13]:
# 6. author that published most books:

i = pd.read_sql_query("""

SELECT a.author_id, a.author_name, COUNT(DISTINCT(b.book_id)) AS 'Books Published'
FROM author a
INNER JOIN book_author ba
ON a.author_id = ba.author_id
INNER JOIN book b
ON ba.book_id = b.book_id 
GROUP BY a.author_id
ORDER BY COUNT(DISTINCT(b.book_id)) DESC
LIMIT 1;

""" , con = conn) 
i

,author_id,author_name,Books Published
0,8222,Stephen King,70


In [14]:
# 7. best seller author

j = pd.read_sql_query("""

SELECT a.author_id, a.author_name, COUNT(ol.order_id) AS Num_Sold 
FROM author a
INNER JOIN book_author ba
ON a.author_id = ba.author_id
INNER JOIN book b
ON ba.book_id= b.book_id
INNER JOIN order_line ol
ON b.book_id = ol.book_id
GROUP BY a.author_id
ORDER BY COUNT(ol.order_id) DESC 
LIMIT 1;

""" , con = conn) 
j

,author_id,author_name,Num_Sold
0,8222,Stephen King,166


In [15]:
# 8. best seller book
# books sold total

k = pd.read_sql_query("""

SELECT COUNT(book_id)
FROM order_line;

""" , con = conn) 
k

,COUNT(book_id)
0,26950


In [16]:
# DISTINCT books sold total

l = pd.read_sql_query("""

SELECT COUNT(DISTINCT(book_id))
FROM order_line;

""" , con = conn) 
l

,COUNT(DISTINCT(book_id))
0,10143


In [17]:
# best seller book

m = pd.read_sql_query("""

SELECT b.book_id, b.title, COUNT(*)
FROM book b
RIGHT JOIN order_line ol
ON b.book_id = ol.book_id
GROUP BY b.book_id
ORDER BY COUNT(*) DESC
LIMIT 10;

""" , con = conn) 
m

,book_id,title,COUNT(*)
0,4369,Personal Injuries (Kindle County Legal Thrille...,10
1,3954,Murder on the Orient Express (Hercule Poirot ...,10
2,2322,Rereading America: Cultural Contexts for Criti...,9
3,10374,Haruki Murakami and the Music of Words,9
4,10507,King Henry IV Part 1,9
5,2514,Roses Are Red (Alex Cross #6),9
6,4912,Discourse on the Origin of Inequality,9
7,857,The Trial,9
8,2719,Life the Universe and Everything (Hitchhiker'...,8
9,661,Mandy,8


In [18]:
# 9. which book bring the most profit:

n = pd.read_sql_query("""

SELECT ol.book_id, b.title, a.author_name, SUM(ol.price) 
FROM order_line ol
LEFT JOIN book b
ON ol.book_id = b.book_id
LEFT JOIN book_author ba
ON b.book_id = ba.author_id
LEFT JOIN author a
ON ba.author_id = a.author_id
GROUP BY ol.book_id 
ORDER BY SUM(ol.price) DESC 
LIMIT 1;

""" , con = conn) 
n

,book_id,title,author_name,SUM(ol.price)
0,5995,Frogs/Assemblywomen/Wealth (Loeb Classical Lib...,Mercedes Lackey,2908.09


In [19]:
# 10. top 5 books that have the most languages versions

o = pd.read_sql_query("""

SELECT b.title, b.book_id, bl.language_name, COUNT(*) 
FROM book b
LEFT JOIN book_language bl
ON b.language_id = bl.language_id
GROUP BY b.book_id
ORDER BY COUNT(DISTINCT(bl.language_name)) DESC 
LIMIT 5;

""" , con = conn) 
o

,title,book_id,language_name,COUNT(*)
0,The World's First Love: Mary Mother of God,1,United States English,1
1,The Illuminati,2,English,1
2,The Servant Leader,3,English,1
3,What Life Was Like in the Jewel in the Crown: ...,4,English,1
4,Cliffs Notes on Aristophanes' Lysistrata The ...,5,English,1


In [20]:
# 11. books languages rank

p = pd.read_sql_query("""

SELECT bl.language_id, bl.language_name, COUNT(*), COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM book_language bl
RIGHT JOIN book b
ON bl.language_id = b.language_id
GROUP BY b.language_id
ORDER BY COUNT(*) DESC;

""" , con = conn) 
p

,language_id,language_name,COUNT(*),Percent of total
0,1,English,8911,80.08448
1,2,United States English,1409,12.66289
2,4,Spanish,218,1.95920
3,5,British English,214,1.92325
4,3,French,144,1.29415
5,10,German,99,0.88973
6,11,Japanese,46,0.41341
7,6,Multiple Languages,19,0.17076
8,14,Chinese,14,0.12582
9,7,Greek,11,0.09886


In [21]:
# 12. which customer bring the most profit

q = pd.read_sql_query("""

SELECT c.customer_id, c.first_name, c.last_name, SUM(ol.price) AS Spent 
FROM customer c
INNER JOIN cust_order co
ON c.customer_id = co.customer_id
INNEr JOIN order_line ol
ON co.order_id = ol.order_id 
GROUP BY c.customer_id
ORDER BY Spent DESC
LIMIT 5;

""" , con = conn) 
q

,customer_id,first_name,last_name,Spent
0,650,Reuben,Usher,841.83
1,752,Seline,Waghorne,740.60
2,846,Godart,Newstead,735.21
3,1233,Tallulah,Janata,674.42
4,1897,Garrick,Rhucroft,672.49


In [22]:
# 13. top 5 cities have most customer placed order

r = pd.read_sql_query("""

SELECT ad.city, ctr.country_name, COUNT(*), COUNT(*) * 100.0 / SUM(COUNT(*)) OVER() AS 'Percent of total'
FROM address ad
INNER JOIN customer_address ca
ON ad.address_id = ca.address_id
INNER JOIN cust_order co
ON ca.customer_id = co.customer_id
INNER JOIN order_line ol
ON co.order_id = ol.order_id
INNER JOIN country ctr
ON ad.country_id = ctr.country_id
GROUP BY ad.city, ctr.country_name
ORDER BY COUNT(*) DESC 
LIMIT 5;

""" , con = conn) 
r

,city,country_name,COUNT(*),Percent of total
0,Miragoâne,Haiti,340,0.60414
1,Bongandanga,Democratic Republic of the Congo,317,0.56328
2,Shangde,China,305,0.54195
3,Burirao,Philippines,256,0.45488
4,Quebradas,Honduras,245,0.43534
